# LangGraph를 활용한 Agent 구축

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

## 도구(Tool) 사용하기
### 검색 API 도구
Tavily 검색 API를 활용하여 검색 기능을 구현하는 도구 
- `TavilySearchResults`
- `TavilyAnswer`

API키 발급 주소: https://app.tavily.com/

발급받은 API키는 환경변수에 설정

In [2]:
from langchain.tools import tool
from typing import List, Dict
from langchain_teddynote.tools import GoogleNews

# 도구 정의 
@tool
def search_news(query: str) -> List[Dict[str, str]]:
    """Search Google News by input keyword"""
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)

print(f"도구 이름: {search_news.name}")
print(f"도구 설명: {search_news.description}")

도구 이름: search_news
도구 설명: Search Google News by input keyword


In [3]:
# tools 정의 
tools = [search_news]

In [4]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages


# State 정의
class State(TypedDict):
    # list 타입에 add_messages 적용(list 에 message 추가)
    messages: Annotated[list, add_messages]

In [5]:
from langchain_ollama import ChatOllama

# LLM 정의, Qwen2.5 7B
llm = ChatOllama(model="qwen2.5:latest")

# LLM에 도구 바인딩
llm_with_tools = llm.bind_tools(tools)

In [6]:
# 노드 함수 정의
def chatbot(state: State):
    answer = llm_with_tools.invoke(state["messages"])
    # 메시지 목록 반환
    return {"messages": [answer]}  # 자동으로 add_messages 적용

In [7]:
from langgraph.graph import StateGraph

# 상태 그래프 초기화
graph_builder = StateGraph(State)

# 노드 추가
graph_builder.add_node("chatbot", chatbot)

In [9]:
import json
from langchain_core.messages import ToolMessage


class BasicToolNode:
    """Run tools requested in the last AIMessage node"""

    def __init__(self, tools: list) -> None:
        # 도구 리스트
        self.tools_list = {tool.name: tool for tool in tools}

    def __call__(self, inputs: dict):
        # 메시지가 존재할 경우 가장 최근 메시지 1개 추출
        if messages := inputs.get("messages", []):
            message = messages[-1]
        else:
            raise ValueError("No message found in input")

        # 도구 호출 결과
        outputs = []
        for tool_call in message.tool_calls:
            # 도구 호출 후 결과 저장
            tool_result = self.tools_list[tool_call["name"]].invoke(tool_call["args"])
            outputs.append(
                # 도구 호출 결과를 메시지로 저장
                ToolMessage(
                    content=json.dumps(
                        tool_result, ensure_ascii=False
                    ),  # 도구 호출 결과를 문자열로 변환
                    name=tool_call["name"],
                    tool_call_id=tool_call["id"],
                )
            )

        return {"messages": outputs}


# 도구 노드 생성
tool_node = BasicToolNode(tools=[tool])

# 그래프에 도구 노드 추가
graph_builder.add_node("tools", tool_node)

AttributeError: 'function' object has no attribute 'name'